# Assessment Problems

Python project that downloads FAANG stock data, plots it, automates execution via a script, and schedules it weekly with GitHub Actions.

## Problem 1: Data from yfinance

Function `get_data()` uses [yfinance](https://github.com/ranaroussi/yfinance) Python package to:

1. Download hourly stock data for the past 5 days for FAANG (META, AAPL, AMZN, NFLX, GOOG).

2. Save results as a CSV in a data/ folder (create it if missing).

The filename format is `YYYYMMDD-HHmmss.csv` (date + time of save).

In [ ]:
# importing yfinance module to fetch stock data
import yfinance

# defining the function to get stock data for a list of ticker symbols
def get_stock_data(ticker_symbols):
    # fetching stock data using yfinance. It returns a Tickers dataframe
    # all functions of yfinance are described at https://ranaroussi.github.io/yfinance/
    stock_df = yfinance.Tickers(ticker_symbols)
    # returning the historical market data for the last 5 days
    return stock_df.history(period="5d")

# downloading stock data for Apple, Microsoft, Google, Amazon, and Tesla 
ticker_symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
stock_data = get_stock_data(ticker_symbols)
# printing the fetched stock data
print(stock_data)


[*********************100%***********************]  5 of 5 completed

Price            Close                                                  \
Ticker            AAPL        AMZN       GOOGL        MSFT        TSLA   
Date                                                                     
2025-10-03  258.019989  219.509995  245.350006  517.349976  429.829987   
2025-10-06  256.690002  220.899994  250.429993  528.570007  453.250000   
2025-10-07  256.480011  221.779999  245.759995  523.979980  433.089996   
2025-10-08  258.059998  225.220001  244.619995  524.849976  438.690002   
2025-10-09  253.679993  227.119995  240.830002  520.945007  434.940002   

Price      Dividends                       ... Stock Splits                  \
Ticker          AAPL AMZN GOOGL MSFT TSLA  ...         AAPL AMZN GOOGL MSFT   
Date                                       ...                                
2025-10-03       0.0  0.0   0.0  0.0  0.0  ...          0.0  0.0   0.0  0.0   
2025-10-06       0.0  0.0   0.0  0.0  0.0  ...          0.0  0.0   0.0  0.0   
2025-10-07  

## Problem 2: Plotting Data

Function `plot_data()` loads the latest CSV in the `data` folder, plots `Close` prices for all five FAANG stocks with labels, legend, and date as the title.
Then it saves the plot in a `plots` folder as `YYYYMMDD-HHmmss.png` by creating the folder if it doesn't exist.

## Problem 3: Script

Script `faang.py` in the repo root contains the two functions (`get_data()` and `plot_data()`). It can be run by `./faang.py` from the terminal to download the data and generate the plot.

## Problem 4: Automation

GitHub Actions workflow (`faang.yml`) inside `.github/workflows/` runs `faang.py` every Saturday morning.

## End